# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""
print("Number of twits in the dataset = {:,}".format(len(twits['data'])))

Number of twits in the dataset = 1,548,010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [5]:
print("Number of messages in the dataset = {:,}".format(len(messages)))
print("Number of sentiments in the dataset = {:,}".format(len(sentiments)))

Number of messages in the dataset = 1,548,010
Number of sentiments in the dataset = 1,548,010


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [8]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    url_pattern = re.compile('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')
    text = re.sub(url_pattern, ' ', text)
   
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    ticker_pattern = re.compile('\$[a-z]+')
    text = re.sub(ticker_pattern, ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    user_name_pattern = re.compile('\@[a-z]+\s')
    text = re.sub(user_name_pattern, ' ', text)

    # Replace everything not a letter with a space
    not_letter_patterm = re.compile('(?:[^a-zA-Z0-9.\s]|[^[0-9]+[\.]*[0-9]+]|([\.]{2,}))')
    text = re.sub(not_letter_patterm, ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [[wnl.lemmatize(word) for word in tokens if len(word) > 1]]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [9]:
#Test the preprocessing by inspecting some output
print(preprocess('$FITB great buy at 26.00...ill wait.'))
print(preprocess('$BAC ok good we&#39;re not dropping in price over the weekend, lol'))

[['great', 'buy', 'at', '26.00', 'ill', 'wait.']]
[['ok', 'good', 'we', '39', 're', 'not', 'dropping', 'in', 'price', 'over', 'the', 'weekend', 'lol']]


In [10]:
#add tqdm so i can track progress
from tqdm import tqdm

tokenized = [preprocess(message) for message in tqdm(messages)]

100%|██████████| 1548010/1548010 [06:29<00:00, 3973.23it/s]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [21]:
from collections import Counter

"""
Create a vocabulary by using Bag of words
"""

bow = Counter()
#Throw away temp variable so i don't see None printed multiple time on the screen
temp = [[bow.update(token) for token in message if len(token) > 1] for message in tqdm(tokenized)]

sorted_bow =  dict(sorted(bow.items(), key=lambda pair: pair[1], reverse=True))


100%|██████████| 1548010/1548010 [00:13<00:00, 112380.00it/s]


In [29]:
print(sorted_bow["bancorp"])

275


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [30]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word: count for word, count in sorted_bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 2.00

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 13

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word, count in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
#filtered_words = [word for word in bow if word not in K_most_common and (random.random() > (1 - p_drop[word]) )]
print(K_most_common)
len(filtered_words) 

['the', 'amp', 'to', 'utm', 'is', 'for', 'on', 'of', 'and', 'in', 'this', '39', 'it']


89796

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [41]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: ii for ii, word in  enumerate(filtered_words)}

#convert to dict to 
vocab = dict(vocab)
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: word  for ii, word in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[[ word for word in message if word in vocab] for message in message_list] for message_list in tqdm(tokenized)]

#filtered = filtered=[[word for word in message if word in vocab] for message in tqdm(tokenized)]




  2%|▏         | 23850/1548010 [00:56<1:00:40, 418.61it/s]




  0%|          | 0/1548010 [00:00<?, ?it/s]




  2%|▏         | 24201/1548010 [00:00<00:06, 242001.58it/s]




  3%|▎         | 47747/1548010 [00:00<00:06, 239999.75it/s]




  5%|▍         | 71448/1548010 [00:00<00:06, 239093.80it/s]




  6%|▌         | 95453/1548010 [00:00<00:06, 239378.56it/s]




  8%|▊         | 120068/1548010 [00:00<00:05, 241369.43it/s]




  9%|▉         | 145560/1548010 [00:00<00:05, 245278.59it/s]




 11%|█         | 170003/1548010 [00:00<00:05, 245021.36it/s]




 13%|█▎        | 193618/1548010 [00:00<00:05, 242082.61it/s]




 14%|█▍        | 218230/1548010 [00:00<00:05, 243277.51it/s]




 16%|█▌        | 242096/1548010 [00:01<00:05, 241869.29it/s]




 17%|█▋        | 268336/1548010 [00:01<00:05, 247681.95it/s]




 19%|█▉        | 293558/1548010 [00:01<00:05, 249025.67it/s]




 21%|██        | 318847/1548010 [00:01<00:04, 250172.23it/s]




 22%|██▏       | 343654/1548010 [00:01<00:04,

In [42]:
print(filtered[:10])

[[['great', 'buy', 'at', '26.00', 'ill', 'wait']], [[]], [['staanalystalert', 'jefferies', 'maintains', 'with', 'rating', 'hold', 'setting', 'target', 'price', 'at', 'usd', '350.00.', 'our', 'own', 'verdict', 'buy', 'toprating']], [['heard', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'stocktwits.']], [['reveal', 'yourself']], [['why', 'drop', 'warren', 'buffet', 'taking', 'out', 'his', 'position']], [['bear', 'have', 'reason', '06', '29', 'pay', 'more', 'attention', 'ba']], [['ok', 'good', 'we', 're', 'not', 'dropping', 'price', 'over', 'weekend', 'lol']], [['daily', 'chart', 'we', 'need', 'get', 'back', 'above', '50.']], [['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'month', 'back', '12', 'if', 'bo', 'then', 'bingo', 'what', 'odds']]]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [43]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [44]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19971918744002004

Finally let's convert our tokens into integer ids which we can pass to the network.

In [45]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

TypeError: unhashable type: 'list'

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [4]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # Setup embedding layer
        #print("vocab_size {}, embed_size {}".format(vocab_size, embed_size))
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)

        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_softmax = nn.LogSoftmax()

    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        weight = next(self.parameters()).data
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        nn_input = nn_input.long()
        #print("nn_input {}".format(nn_input))
                
        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
                
        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # softmax function
        soft_out = self.log_softmax(out)
        
        return soft_out, hidden_state

### View Model

In [5]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
print(model)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
print(input.size())
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

NameError: name 'vocab' is not defined

## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [23]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [24]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

split_frac = 0.8
split_idx = int(len(token_ids)*0.8)

train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

print('train_features:', len(train_features))
print('train_labels:', len(train_labels))

train_features: 832690
train_labels: 832690


In [25]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)

print(model)
print(text_batch.size())
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

TextClassifier(
  (embedding): Embedding(90315, 200)
  (lstm): LSTM(200, 128)
  (dropout): Dropout(p=0.0)
  (fc): Linear(in_features=128, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)
torch.Size([20, 64])


### Training
It's time to train the neural network!

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

cuda


TextClassifier(
  (embedding): Embedding(90315, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [69]:
import numpy as np
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 1024
learning_rate = 0.001
train_loss = 0
clip = 5
seq_len = 40

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        #print("Training Text Batch Size....... = ",text_batch.size())
        #print("Training Labels Size....... = ",labels.size())
        
        # initialize hidden state
        
        steps += 1
        #hidden = tuple([each.data for each in hidden])
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        #print("Training Text Batch Size....... = ",text_batch.size())
        #print("Training Labels Size....... = ",labels.size())
        
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history  
        hidden = tuple([each.data for each in hidden])
        
        # zero accumulated gradients
        model.zero_grad()
        
        # get the output from the model
        logps, hidden = model.forward(text_batch, hidden)
        
        # calculate the loss and perform backprop
        loss = criterion(logps, labels)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        
        if steps % print_every == 0:
            # Get validation loss
            model.eval()
            
            val_losses = []
            accuracy = []
           
            with torch.no_grad():
            
                for inputs, labels in dataloader(
                    valid_features, valid_labels, batch_size=batch_size, sequence_length=seq_len, shuffle=True):
                    

                    val_h = model.init_hidden(labels.shape[0])

                     # Set Device
                    inputs, labels = inputs.to(device), labels.to(device)
                    for each in val_h:
                        each.to(device)

                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history    
                    val_h = tuple([each.data for each in val_h])

                    logps, val_h = model.forward(inputs, val_h)
                    
                    val_loss = criterion(logps, labels)

                    val_losses.append(val_loss.item()) 

                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())

               
            model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
              "Steps: {}...".format(steps),
              "Train Loss: {:.6f}...".format(loss.item()),
              "Val Loss: {:.6f}".format(np.mean(val_losses)),
              "Accuracy: {:.6f}".format(np.mean(accuracy)))
            #print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')
            

Starting epoch 1
tensor([[ -3.4968,  -1.7045,  -3.1355,  -1.0645,  -0.9177],
        [ -0.5047,  -2.6023,  -2.9290,  -1.3980,  -3.8307],
        [ -3.8954,  -0.1341,  -2.8117,  -3.1252,  -6.7633],
        ...,
        [ -2.6653,  -1.8685,  -1.1853,  -0.9217,  -2.6229],
        [ -2.4251,  -0.1739,  -4.0515,  -2.9522,  -6.5062],
        [ -6.3726,  -5.0799,  -5.3513,  -2.9954,  -0.0647]], device='cuda:0')
tensor([[ -4.0172,  -2.5283,  -0.5284,  -1.2339,  -3.8405],
        [ -2.5265,  -3.5823,  -2.1038,  -1.7965,  -0.5035],
        [ -7.0415,  -0.0304,  -3.8238,  -4.9319,  -9.8382],
        ...,
        [ -8.3784,  -5.6747,  -4.5751,  -0.0192,  -5.2804],
        [ -1.8664,  -1.1696,  -1.4060,  -1.4413,  -2.9357],
        [ -2.0202,  -2.1081,  -1.1410,  -1.3543,  -1.7821]], device='cuda:0')
tensor([[ -7.1302,  -5.3726,  -5.9576,  -4.4626,  -0.0198],
        [ -5.8338,  -4.8255,  -0.0381,  -3.7953,  -5.5407],
        [ -2.8671,  -3.1188,  -0.7062,  -1.1940,  -2.2789],
        ...,
        

KeyboardInterrupt: 

In [29]:
torch.save(model, "ni_model.pt")

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type TextClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [6]:
# Model class must be defined somewhere
model = torch.load("ni_model.pt")

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at torch/csrc/cuda/Module.cpp:51

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [91]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    model.eval()
    
    tokens = preprocess(text)
        
    # Filter non-vocab words
    tokens = [[word for word in message if word in vocab] for message in tqdm(tokens)]
    
    
    # Convert words to ids
    tokens = [vocab[word] for word in message for message in tokens]
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(np.asarray(tokens))
    
    # Adding a batch dimension
    text_input = feature_tensor.size(0)
 
    # Get the NN output
    # initialize hidden state
    hidden = model.init_hidden(text_input)
    
    logps, _  = model.forward(feature_tensor.unsqueeze(0), hidden) 
    print(logps)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.round(logps.squeeze()) 
      
    ps = torch.exp(logps)
    
    top_p, top_class = ps.topk(1, dim=0)
    print(top_class)
    
    
    # print custom response
    if(top_class==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")      
    
    return pred

In [94]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")

predict(text, model, vocab)

100%|██████████| 1/1 [00:00<00:00, 5440.08it/s]

tensor([[-2.1375, -1.7600, -1.2711, -1.6181, -1.4645],
        [-1.1601, -1.7889, -2.1469, -2.0705, -1.2858],
        [-2.1034, -2.1044, -1.6418, -1.6992, -0.9687],
        [-2.7528, -1.9928, -1.9664, -0.7470, -1.6809],
        [-2.3844, -1.6660, -1.0810, -1.1390, -2.8228],
        [-2.2646, -1.7121, -1.2615, -0.9658, -2.9619]])
tensor([[ 1,  4,  4,  3,  2]])


RuntimeError: bool value of Tensor with more than one value is ambiguous

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [ ]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [ ]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

Using the `prediction` function, let's apply it to a stream of twits.

In [ ]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [ ]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.